In [1]:
!pip install docplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docplex: filename=docplex-2.25.236-py3-none-any.whl size=671350 sha256=6b67244d1ad45388aae669f2eb795f80b9ea32e2f527bbc5054279972e82341e
  Stored in directory: /root/.cache/pip/wheels/3b/e5/00/0bf0173d67188fe73a13e3a61412b3f975f60205e3fab93a69
Successfully built docplex


In [ ]:
#!pip install pulp

In [ ]:
#!pip install gurobipy

In [ ]:
import pandas as pd
import numpy as np
#import gurobipy as gp
#from gurobipy import GRB
from pulp import *

In [ ]:
data = pd.read_csv('./data_hr.csv')

In [ ]:
data['Delay'].sum()

2.7

In [ ]:
data = data.iloc[6:14]

In [ ]:
data = data.reset_index(drop = True)

In [ ]:
mip = LpProblem("BACAP", LpMinimize)

In [ ]:
#Paramaters
N = 8      #total vessel number
gap_y = 20  #distance interval(unit = meter)
gap_x = 1  #time interval(unit = minute)
T = 108  #planning period
L = 800     #quay length
Q = 7       #total QC numbers
qmin = 1    #minimum QC/hr for vessel i
qmax = 4    #maximum QC/hr for vessel i
p = 25      #productivity

time_epoch = 12    #rescheduling epoch(minute)
M = 999999        #big M

In [ ]:
r = np.array(data['Total'])     #total container numbers
cost = 15*r                   #penalty cost
m = r/p                         #total number of QC required for vessel i
ETA = np.array(data['ETB']).astype(int)     #ETA for vessel i
ETD = np.array(data['ETD'])     #ETD for vessel i
l = np.array(data['배길이(m)']) #vessel length

In [ ]:
v_x = {(i, j): LpVariable(f"v_x_{i}_{j}", cat="Binary")
       for i in range(N) for j in range(N) if i != j}
v_y = {(i, j): LpVariable(f"v_y_{i}_{j}", cat="Binary")
       for i in range(N) for j in range(N) if i != j}
b = {i: LpVariable(f"b_{i}", lowBound=0) for i in range(N)}
x = {i: LpVariable(f"x_{i}", lowBound=0) for i in range(N)}
d = {i: LpVariable(f"d_{i}", lowBound=0) for i in range(N)}

In [ ]:
q = [[[ LpVariable(f"q_{i}{t}{n}", cat="Binary") for n in range(qmin, qmax+1)] for t in range(18)] for i in range(N)]

In [ ]:
#v_x

In [ ]:
#q

In [ ]:
#set objectives
mip.setObjective(lpSum(cost[i] * max(d[i] - ETD[i]) for i in range(N)))

# constraint

In [ ]:
import math

In [ ]:
ETA

array([30, 50, 53, 56, 82, 82, 84, 97])

In [ ]:
## 새로운 constraint 구현

V1 = [0,1,2,3,4]  # list of jobs in V^1
V2 = [5,6,7]  # list of jobs in V^2
b_1 = [0] * N
x_1 = [0] * N
tau = 9
# define constraints
for i in V1:
    mip += b[i] == b_1[i] #2
    mip += x[i] == x_1[i] #3
for i in V2:
    mip += b[i] >= ETA[i] #4
    mip += x[i] <= L - l[i] #5
    for t in range(max(ETA[i], tau), T+1):
        mip += b[i] <= t * lpSum([q for n in range(qmin, qmax+1)]) + M * (1 - lpSum([q for n in range(qmin, qmax+1)])) #9
        mip += d[i] <= t * lpSum([q for n in range(qmin, qmax+1)]) + 1 #10
for i in range(N):
    for j in range(N):
        if i != j:
            mip += b[j] >= d[i] + gap_x + M * (v_x[(i, j)] - 1) #6
            mip += x[j] >= x[i] + l[i] + gap_y + M * (v_y[(i, j)] - 1) #7
            mip += v_x[(i, j)] + v_x[(j, i)] + v_y[(i, j)] + v_y[(j, i)] >= 1 #8
    mip += d[i] - b[i] <= math.ceil(m[i] / qmin) #11
#for t in range(tau, T+1):
    #mip += lpSum([n * q[(i, t, n)] for i in V1 for n in range(qmin, qmax+1)]) >= m_1 #12 m_1 정의 필요
    #mip += lpSum([n * q[(i, t, n)] for i in V2 for n in range(qmin, qmax+1)]) >= m #13
    #mip += lpSum([[n * q[(i, t, n)] for i in range(N)] for n in range(qmin, qmax+1)]) <= Q #14
    #mip += lpSum([[q[(i, t, n)] for i in range(N)] for n in range(qmin, qmax+1)]) <= 1 #15

for i in range(N):
    for t in range(tau,T+1):
        for n in range(qmin,qmax+1):
            mip += lpSum(n * q) <= Q
            mip += lpSum(q) <= 1
                   



In [ ]:
mip.solve()

-1

In [ ]:
mip.value(d)

AttributeError: 'LpProblem' object has no attribute 'value'

In [ ]:
v = set()
v1 = set()
v2 = set()

for i in range(N):
  v.add(i)

In [ ]:
for i in v2:
  mip += b[i] >= ETA[i]
  mip += x[i] <= L-l[i]

for i in range(N):
  for j in range(N):
    if i ==j:
      continue
      mip += b[j] >= d[i] + gap_x + M*(vx[i,j] - 1)
      mip += x[j] >= x[i] + l[i] + gap_y + M*(vy[i,j]-1)
      mip += vx[i,j] + vx[j,i] + vy[i,j] +vy[j,i] >= 1

for i in range(N):
  mip += d[i] - b[i] <= math.ceil(m[i]/qmin)

for i in v1:
  mip += xsum(q * qitn_i[i,j,q] for j in range(18) for q in range(qmin, qmax+1)) >= m[i]

for i in v2:
  mip += xsum(q * qitn_i[i,j,q] for j in range(18) for q in range(qmin, qmax+1)) >= m[i]

mip += xsum(q * qitn_i[i, j, q] for i in v for j in range(18) for q in range(qmin, qmax+1)) <= Q

for i in range(N):
  mip += x[i] >= 0
  mip += b[i] >= 0
  mip += d[i] >= 0
  mip += y[i] >= 0
  mip += y[i] >= d[i] - ETD[i]

In [ ]:
mip.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [ ]:
mip.objective_value

0.0

In [ ]:
mip.display

AttributeError: ignored

In [ ]:
for i in range(N):
  print('departure time[',i,']=',d[i].x)
  print('berthing position[',i,']=',x[i].x)
  print('berthing time[',i,']=',b[i].x)
  print()

departure time[ 0 ]= 0.0
berthing position[ 0 ]= 0.0
berthing time[ 0 ]= 0.0

departure time[ 1 ]= 0.0
berthing position[ 1 ]= 0.0
berthing time[ 1 ]= 0.0

departure time[ 2 ]= 0.0
berthing position[ 2 ]= 0.0
berthing time[ 2 ]= 0.0

departure time[ 3 ]= 0.0
berthing position[ 3 ]= 0.0
berthing time[ 3 ]= 0.0

departure time[ 4 ]= 0.0
berthing position[ 4 ]= 0.0
berthing time[ 4 ]= 0.0

departure time[ 5 ]= 0.0
berthing position[ 5 ]= 0.0
berthing time[ 5 ]= 0.0

departure time[ 6 ]= 0.0
berthing position[ 6 ]= 0.0
berthing time[ 6 ]= 0.0

departure time[ 7 ]= 0.0
berthing position[ 7 ]= 0.0
berthing time[ 7 ]= 0.0

departure time[ 8 ]= 0.0
berthing position[ 8 ]= 0.0
berthing time[ 8 ]= 0.0

departure time[ 9 ]= 0.0
berthing position[ 9 ]= 0.0
berthing time[ 9 ]= 0.0

departure time[ 10 ]= 0.0
berthing position[ 10 ]= 0.0
berthing time[ 10 ]= 0.0

departure time[ 11 ]= 0.0
berthing position[ 11 ]= 0.0
berthing time[ 11 ]= 0.0

departure time[ 12 ]= 0.0
berthing position[ 12 ]= 0.0
ber